Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-06 05:34:06--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  69.5MB/s    in 2.8s    

2019-11-06 05:34:09 (69.5 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
# Imports
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [0]:
# Load Orders, Products, and OrderProducts into dataframes
df_orders = pd.read_csv('orders.csv')
df_products = pd.read_csv('products.csv')
df_order_products_prior = pd.read_csv('order_products__prior.csv')
df_order_products_train = pd.read_csv('order_products__train.csv')

# Print out the columns in each dataframe so we can inspect them for use downstream
print(f'Orders Columns: {list(df_orders.columns)}')
print(f'Products Columns: {list(df_products.columns)}')
print(f'Order Products Columns: {list(df_order_products_prior.columns)}')


Orders Columns: ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
Products Columns: ['product_id', 'product_name', 'aisle_id', 'department_id']
Order Products Columns: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']


In [0]:
# Concatenate the order_product datasets to get a full picture of all product orders
df_order_products_all = pd.concat([df_order_products_prior, df_order_products_train])
df_order_products_all.sample()

,order_id,product_id,add_to_cart_order,reordered
12865735,1358080,47865,12,0


In [0]:
# Produce an interim dataframe of product names - include a "normalized" product name that:
#  - is lowercase
#  - strips out "whitespace"

# "nrmliz" is a function that returns a string that is lowercase and stripped of whitespace
def nrmliz(str):
  tmp = re.sub('\s','', str)
  tmp = tmp.lower()
  return tmp

# Create a column of normalized product names by applying the "nrmliz" function
df_products['product_name_clean'] = df_products['product_name'].apply(nrmliz)


In [0]:
# Do a quick and dirty check to see if we have duplicate product names (due to weird spellngs, whitespace)
print('banana', len(df_products[df_products['product_name_clean'] == 'banana']))
print('bagoforganicbananas', len(df_products[df_products['product_name_clean'] == 'bagoforganicbananas']))
print('organicstrawberries', len(df_products[df_products['product_name_clean'] == 'organicstrawberries']))
print('organicbabyspinach', len(df_products[df_products['product_name_clean'] == 'organicbabyspinach']))
print('organichassavocado', len(df_products[df_products['product_name_clean'] == 'organichassavocado']))
print('organicavocado', len(df_products[df_products['product_name_clean'] == 'organicavocado']))
print('largelemon', len(df_products[df_products['product_name_clean'] == 'largelemon']))
print('strawberries', len(df_products[df_products['product_name_clean'] == 'strawberries']))
print('limes', len(df_products[df_products['product_name_clean'] == 'limes']))
print('organicwholemilk', len(df_products[df_products['product_name_clean'] == 'organicwholemilk']))

# Do we have multiple 'Organic Whole Mile' products?
print(f'\n{df_products[df_products["product_name_clean"] == "organicwholemilk"]}')


banana 1
bagoforganicbananas 1
organicstrawberries 1
organicbabyspinach 1
organichassavocado 1
organicavocado 1
largelemon 1
strawberries 1
limes 1
organicwholemilk 2

       product_id         product_name  ...  department_id  product_name_clean
27844       27845   Organic Whole Milk  ...             16    organicwholemilk
31719       31720  Organic  Whole Milk  ...             16    organicwholemilk

[2 rows x 5 columns]


### Product Name Findings

* 'Organic Whole Milk' is represented by two product ids.  We may need to use the "cleaned" or normalized product names.

### Column Selection

What columns should we select to include in a single dataframe?

* `order_id` from the `df_order_products_all` dataframe
* `product_name_clean` from the `df_products` dataframe
* `product_name` from the `df_products` dataframe
*  `product_id` from the `df_products` dataframe


In [0]:
# Merge the df_products and df_order_products_all dataframes
df_count_join = df_order_products_all.merge(df_products)

df_count_join.sample()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,product_name_clean
13236305,2065863,46720,14,1,Organic Free Range Chicken Broth,69,15,organicfreerangechickenbroth


In [0]:
# Count up the orders of the most frequently ordered products
# Construct a list of our high frequency products
top_products = [
    {'name': 'Banana', 'normal': 'banana'},
    {'name': 'Bag of Organic Bananas', 'normal': 'bagoforganicbananas'},
    {'name': 'Organic Strawberries', 'normal': 'organicstrawberries'},
    {'name': 'Organic Baby Spinach', 'normal': 'organicbabyspinach'},
    {'name': 'Organic Hass Avocado', 'normal': 'organichassavocado'},
    {'name': 'Organic Avocado', 'normal': 'organicavocado'},
    {'name': 'Large Lemon', 'normal': 'largelemon'},
    {'name': 'Strawberries', 'normal': 'strawberries'},
    {'name': 'Limes', 'normal': 'limes'},
    {'name': 'Organic Whole Milk', 'normal': 'organicwholemilk'}
]

# Do we have any duplicates? (Orders in which the same product is "rung" up more than once)
# Create a dataframe of just just order_id and product_name_clean (normalized product name) 
df_check_multiple_rings = df_count_join[['order_id', 'product_name_clean']]

# Are there any duplicates? If so, these are multiple product "rings" and let's ignore those.
print(f'Number of multiple "rings": {len(df_check_multiple_rings)-len(df_check_multiple_rings.drop_duplicates())}')

# Create a new dataframe that has the multiple rings or duplicate order_id / normalized product name pairs removed
df_no_multiple_rings = df_check_multiple_rings.drop_duplicates()


Number of multiple "rings": 653


In [0]:
# Determine the counts representing the frequency of a product order
# As a convenience, iterate through the top products and count the number of orders using the normalized product name (damn you 'Organic Whole Milk'!)
for prod in top_products:
  print(f'Number of times product "{prod["name"]}" was ordered: {len(df_no_multiple_rings[df_no_multiple_rings["product_name_clean"] == prod["normal"]])}')

Number of times product "Banana" was ordered: 491291
Number of times product "Bag of Organic Bananas" was ordered: 394930
Number of times product "Organic Strawberries" was ordered: 275577
Number of times product "Organic Baby Spinach" was ordered: 251705
Number of times product "Organic Hass Avocado" was ordered: 220877
Number of times product "Organic Avocado" was ordered: 184224
Number of times product "Large Lemon" was ordered: 160792
Number of times product "Strawberries" was ordered: 149445
Number of times product "Limes" was ordered: 146660
Number of times product "Organic Whole Milk" was ordered: 152748


## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [38]:
# Complete Cell 
# Construct table2
table2 = pd.DataFrame(
    [[np.nan, 16, 3],
     [2, 11, 1]], 
    index=['treatmenta', 'treatmentb'], 
    columns=['John Smith', 'Jane Doe', 'Mary Johnson'])

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16,3
treatmentb,2.0,11,1


In [39]:
# Convert table2's index into a column
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16,3
1,treatmentb,2.0,11,1


In [40]:
# Convert table2 from 'wide' to 'tidy' using the melt method
tidy2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe',	'Mary Johnson'])
tidy2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [41]:
# Rename tidy2's columns to something meaningful
tidy2 = tidy2.rename(columns={
    'index': 'treatment',
    'variable': 'name',
    'value': 'result'
})
tidy2

,treatment,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [42]:
# Translate tidy2 (tidy format of table2) back to the original format using pivot_table
table2A = tidy2.pivot_table(index='treatment', columns='name', values='result')
table2A

name,Jane Doe,John Smith,Mary Johnson
treatment,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


### Pivot Table of Seaborn flights Data
Load seaborn's flights dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [43]:
flights.sample(5)

,year,month,passengers
30,1951,July,199
118,1958,November,310
12,1950,January,115
34,1951,November,146
60,1954,January,204


In [44]:
# Generate a pivot table of the Seaborn 'flights' dataset.  Aggregate multiple Year/Month data by summing values
df_flt_pvt = flights.pivot_table(values='passengers', index='year', columns='month', aggfunc='sum')
df_flt_pvt

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####